In [1]:
import pandas as pd
from shapely.geometry import Point
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
from tqdm.auto import tqdm
from folium.plugins import MarkerCluster
from scipy.spatial import distance
import branca.colormap as cm
from folium.plugins import TimeSliderChoropleth
import copy
from scipy.stats import kendalltau

In [2]:
polution_data = pd.read_csv("../data/no2_aggregated.csv")
polution_data['timestamp'] = pd.to_datetime(polution_data['timestamp'])
polution_data = polution_data.set_index('timestamp')
polution_data = polution_data.resample('Y')
polution_data = polution_data.median()

In [3]:
polution_data['time_unix'] = polution_data.index.view(int) / 10**9
polution_data.reset_index(drop=True, inplace=True)

In [4]:
polution_data = pd.melt(polution_data, id_vars=['time_unix'], value_vars=['DsWrocWybCon', 'DsWrocAlWisn', 'DsWrocBartni'],
       var_name='station', value_name='polution')

In [5]:
stations = {
    'DsWrocAlWisn': [51.086225, 17.012689],
    'DsWrocWybCon': [51.129378, 17.029250],
    'DsWrocBartni': [51.115933, 17.141125],
    
    #'DsWrocNaGrob': [51.103456, 17.059225]
}

stations_points = [
    ('DsWrocAlWisn', Point([17.012689, 51.086225])),
    ('DsWrocWybCon', Point([17.029250, 51.129378])),
    ('DsWrocBartni', Point([17.141125, 51.115933])),
    
    #('DsWrocNaGrob', Point([17.059225, 51.103456]))
]

stations_gdf = gpd.GeoDataFrame(stations_points, columns=['station', 'geometry'])
stations_gdf.crs = "epsg:4326"
stations_gdf = stations_gdf.to_crs(epsg=32631) 

polution_data = polution_data.merge(stations_gdf, on='station')

In [6]:
gdf = gpd.read_file('../data/wroclaw.json')
id_district_map = dict(zip(gdf.id.values, gdf.osiedle.values))
gdf = gdf.to_crs(epsg=32631) 

In [7]:
def get_polution(stations_df, centroid):
    values = []
    distances = []
    
    min_dist = float('inf')
    best_pol = 0
    
    for _, row in stations_df.iterrows():
        dist = row.geometry.distance(centroid)
        distances.append((1 / dist))
        values.append(row.polution)
        
        if dist < min_dist:
            min_dist = dist
            best_pol = row.polution
    
    weighted_mean = np.average(values, weights=distances)
    return weighted_mean + 0.1 * best_pol

In [8]:
merged_df = []
for _, row in gdf.iterrows():
    
    for time_point in polution_data.time_unix.unique():
        copied_row = copy.deepcopy(row)
        copied_row['polution'] = get_polution(polution_data[polution_data.time_unix == time_point], row.geometry.centroid)
        copied_row['time_unix'] = time_point
 
        merged_df.append(copied_row)
joined_df = pd.DataFrame(merged_df)

In [9]:
df1 = copy.deepcopy(joined_df)

In [10]:
max_colour = max(joined_df['polution'])
min_colour = min(joined_df['polution'])

cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
joined_df['colour'] = joined_df['polution'].map(cmap)

In [11]:
style_dict = {}
for idx in gdf.id.unique():

    result = joined_df[joined_df['osiedle'] == id_district_map[idx]]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[int(r['time_unix'])] = {'color': r['colour'], 'opacity': 0.6}
    style_dict[str(idx)] = inner_dict

In [12]:
countries_df = joined_df[['id', 'geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_gdf = countries_gdf.drop_duplicates().set_index(['id'])
countries_gdf.crs = "EPSG:32631"
countries_gdf = countries_gdf.to_crs(epsg=4326) 

In [13]:
mapObj = folium.Map(location=[51.12, 17.02], zoom_start=12)

for station, coords in stations.items():
    icon = folium.Icon(icon='cloud', color='red')
    folium.Marker(coords, popup=station, tooltip='Stacja pomiarowa', icon=icon).add_to(mapObj)

_ = TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict,
    name="Zanieczyszczenie powietrza",
    show=True
).add_to(mapObj)

_ = cmap.add_to(mapObj)
cmap.caption = "Poziom zanieczyszczenia"

In [14]:
#folium.LayerControl().add_to(mapObj)
mapObj.save("jakosc_powietrza.html")

## piece

In [15]:
df = pd.read_csv("../data/piece.csv", index_col=0)

df['2015_cum'] = df['2014'] + df['2015']
df['2016_cum'] = df['2015_cum'] + df['2016']
df['2017_cum'] = df['2016_cum'] + df['2017']
df['2018_cum'] = df['2017_cum'] + df['2018']
df['2019_cum'] = df['2018_cum'] + df['2019']
df['2020_cum'] = df['2019_cum'] + df['2020']
df['2021_cum'] = df['2020_cum'] + df['2021']

df['geometry'] = gpd.points_from_xy(df.lon, df.lat)
#df = df[['geometry', '2015_cum', '2016_cum', '2017_cum', '2018_cum', '2019_cum', '2020_cum', '2021_cum']]

In [16]:
df = pd.melt(df, id_vars=['geometry'], value_vars=['2015_cum', '2016_cum', '2017_cum', '2018_cum', '2019_cum', '2020_cum', '2021_cum'],
       var_name='time_unix', value_name='changed')
df['time_unix'] = df['time_unix'].apply(lambda val: val[:-4] + "-12-31", 1)
df['time_unix'] = pd.to_datetime(df['time_unix'])
df['time_unix'] = df['time_unix'].view(int) / 10**9
df = df[df.changed > 0]
#df.changed = np.log2(df.changed)
df = gpd.GeoDataFrame(df)
df.crs = "EPSG:4326"
df = df.to_crs(epsg=32631) 

In [17]:
gdf = gpd.read_file('../data/wroclaw.json')
id_district_map = dict(zip(gdf.id.values, gdf.osiedle.values))
gdf = gdf.to_crs(epsg=32631) 

In [18]:
merged_df = []
for _, row in tqdm(gdf.iterrows(), total=len(gdf)):

    for time_point in df.time_unix.unique():
        changed_here = 0
        copied_row = copy.deepcopy(row)
        copied_row['time_unix'] = time_point
        for _, furnice_row in df[df.time_unix == time_point].iterrows():        
            if row.geometry.contains(furnice_row.geometry):
                changed_here += furnice_row.changed
        
            copied_row['polution'] = changed_here
            merged_df.append(copied_row)
joined_df = pd.DataFrame(merged_df)

  0%|          | 0/48 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
max_colour = max(joined_df['polution'])
min_colour = min(joined_df['polution'])

cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
joined_df['colour'] = joined_df['polution'].map(cmap)

In [ ]:
style_dict = {}
for idx in tqdm(gdf.id.unique()):

    result = joined_df[joined_df['osiedle'] == id_district_map[idx]]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[int(r['time_unix'])] = {'color': r['colour'], 'opacity': 0.6}
    style_dict[str(idx)] = inner_dict

In [ ]:
countries_df = joined_df[['id', 'geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_gdf = countries_gdf.drop_duplicates().set_index(['id'])
countries_gdf.crs = "EPSG:32631"
countries_gdf = countries_gdf.to_crs(epsg=4326) 

In [ ]:
mapObj = folium.Map(location=[51.12, 17.02], zoom_start=12)

_ = TimeSliderChoropleth(
    data=countries_gdf.to_json(),
    styledict=style_dict,
    name="Trend zmiany pieców",
    show=True
).add_to(mapObj)

_ = cmap.add_to(mapObj)
cmap.caption = "Liczba wymienionych pieców"

## pins

In [ ]:
df = pd.read_csv("../data/piece.csv", index_col=0)
mCluster = MarkerCluster(name="Adresy wymienionych pieców", show=True).add_to(mapObj)
for index, row in df.iterrows():
    number = row['all']
    if number > 0:
        icon = folium.Icon(icon='home')
        folium.Marker([row.lat, row.lon], popup=f"Wymienione piece: {number}", tooltip=row.address, icon=icon).add_to(mCluster)

In [ ]:
folium.LayerControl().add_to(mapObj)
mapObj.save("zmienione_piece.html")